In [ ]:
import json
from itertools import combinations
from numbers import Number
from random import shuffle
from collections import Counter
import torch
import time
import logging
import argparse
import os
import math
import random
import numpy as np
import pickle as pkl
from pandas import DataFrame
from sklearn.metrics import roc_auc_score, average_precision_score
from torch import nn

from dataloader import *
import argparse
from transform import *
from captum.attr import IntegratedGradients

with open('../../../inputs_new.pickle','rb') as handle:
    inputs = pkl.load(handle)
print(inputs)
cols = inputs.preops.columns
#get rid of first and last
cols = cols[1:-1]    
#change column name orlogid_encoded to orlogid
inputs.outcomes.rename(columns={'orlogid_encoded':'orlogid'}, inplace=True)
inputs.texts.rename(columns={'orlogid_encoded':'orlogid'}, inplace=True)
print(inputs.outcomes.columns)
folder = ('../preops_cv/')

outcomes = ['cardiac', 'AF','arrest', 'DVT_PE', 'post_aki_status', 'total_blood']
for idx, outcome in enumerate(outcomes):
    foldername = folder+'arrest'+'/'
    aurocs = []
    auprcs = []
    for i in range(5):

        X_train = pkl.load(open(foldername + 'X_train_' + str(i) + '.pickle', 'rb'))
        X_test = pkl.load(open(foldername + 'X_test_' + str(i) + '.pickle', 'rb'))

        ids_train = pkl.load(open('../preops_cv/arrest/' + 'train_ids_' + str(i) + '.pickle', 'rb'))
        ids_test = pkl.load(open('../preops_cv/arrest/' + 'test_ids_' + str(i) + '.pickle', 'rb'))
        train_outcome2 = inputs.outcomes[(inputs.outcomes.orlogid.isin(ids_train))].copy()
        train_outcome2 = train_outcome2[['cardiac', 'AF', 'arrest', 'DVT_PE', 'post_aki_status', 'total_blood']].to_numpy()
        train_outcome2 = np.where(train_outcome2> 0, 1, 0)
        test_outcome2 = inputs.outcomes[(inputs.outcomes.orlogid.isin(ids_test))].copy()
        test_outcome2 = test_outcome2[['cardiac', 'AF', 'arrest', 'DVT_PE', 'post_aki_status', 'total_blood']].to_numpy()
        test_outcome2 = np.where(test_outcome2> 0, 1, 0)
        y_train = train_outcome2[:, idx]
        y_test = test_outcome2[:, idx]

        print(outcome + ' ' + "fold " + str(i))

        vae_model = pkl.load(open('./distangle_vae_model/model_fold' + str(i) + '.pickle', 'rb'))

        X_test = np.delete(X_test, 163, axis=1)

        ''' #print features
        for m in range(len(X_test)):
            for feature, col in zip(X_test[m], cols):
                print(col, feature)'''
        X_test = X_test.astype(np.float32)
        # log transform

        X_test = np.log(X_test + 1)
        vae_model = vae_model.to("cuda")

        #construct model using encoder of vae and classifier
        class VAE(nn.Module):
            def __init__(self, vae_model):
                super(VAE, self).__init__()
                self.feature_encoder = vae_model.feature_encoder
                self.encoder = vae_model.encoder
                self.classifier = vae_model.classifiers[idx]
            def forward(self, x):
                x = self.feature_encoder(x)
                x = self.encoder(x)
                x = self.classifier(x)
                #softmax
                x = nn.Softmax(dim=1)(x[:, :2])
                return x
        model = VAE(vae_model)
        model = model.to("cuda")
        model.train()

        ig = IntegratedGradients(model)
        x_test = torch.from_numpy(X_test).to("cuda")
        # as float32
        x_test = x_test.float()

        targets = torch.tensor(y_test).to("cuda")
        #use batch size of 64
        dataset = torch.utils.data.TensorDataset(x_test, targets)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False)
        #use integrated gradients
        atts = []
        for batch in dataloader:
            x, y = batch
            x.requires_grad = True
            attributions = ig.attribute(x, target=y, n_steps=100)
            atts.append(attributions.cpu().detach().numpy())
            #release memory
            del x
            del y
            del attributions

        #release memory
        del x_test
        del targets
        del dataset
        del dataloader

        atts = np.concatenate(atts, axis=0)

        #save attributions
        if not os.path.exists('./interpret'):
            os.makedirs('./interpret')
        if not os.path.exists('./interpret' + '/attributions_fold_'+ outcome):
            os.makedirs('./interpret' + '/attributions_fold_'+ outcome)
        pkl.dump(atts, open('./interpret' + '/attributions_fold_'+ outcome + '/' + str(i) + '.pickle', 'wb'))

    
    


In [ ]:
import json
from itertools import combinations
from numbers import Number
from random import shuffle
from collections import Counter
import torch
import time
import logging
import argparse
import os
import math
import random
import numpy as np
import pickle as pkl
from pandas import DataFrame
from sklearn.metrics import roc_auc_score, average_precision_score
from torch import nn

from dataloader import *
import argparse
from transform import *
from captum.attr import IntegratedGradients

with open('../../../inputs_new.pickle','rb') as handle:
    inputs = pkl.load(handle)
print(inputs)
cols = inputs.preops.columns
#get rid of first and last
cols = cols[1:-1]    
#change column name orlogid_encoded to orlogid
inputs.outcomes.rename(columns={'orlogid_encoded':'orlogid'}, inplace=True)
inputs.texts.rename(columns={'orlogid_encoded':'orlogid'}, inplace=True)
print(inputs.outcomes.columns)
folder = ('../preops_cv/')

outcomes = ['cardiac', 'AF','arrest', 'DVT_PE', 'post_aki_status', 'total_blood']
for idx, outcome in enumerate(outcomes):
    foldername = folder+'arrest'+'/'
    atts = []
    for i in range(5):
        X_train = pkl.load(open(foldername + 'X_train_' + str(i) + '.pickle', 'rb'))
        X_test = pkl.load(open(foldername + 'X_test_' + str(i) + '.pickle', 'rb'))

        ids_train = pkl.load(open('../preops_cv/arrest/' + 'train_ids_' + str(i) + '.pickle', 'rb'))
        ids_test = pkl.load(open('../preops_cv/arrest/' + 'test_ids_' + str(i) + '.pickle', 'rb'))
        train_outcome2 = inputs.outcomes[(inputs.outcomes.orlogid.isin(ids_train))].copy()
        train_outcome2 = train_outcome2[['cardiac', 'AF', 'arrest', 'DVT_PE', 'post_aki_status', 'total_blood']].to_numpy()
        train_outcome2 = np.where(train_outcome2> 0, 1, 0)
        test_outcome2 = inputs.outcomes[(inputs.outcomes.orlogid.isin(ids_test))].copy()
        test_outcome2 = test_outcome2[['cardiac', 'AF', 'arrest', 'DVT_PE', 'post_aki_status', 'total_blood']].to_numpy()
        test_outcome2 = np.where(test_outcome2> 0, 1, 0)
        y_train = train_outcome2[:, idx]
        y_test = test_outcome2[:, idx]

        print(outcome + ' ' + "fold " + str(i))
        vae_model = pkl.load(open('./distangle_vae_model/model_fold' + str(i) + '.pickle', 'rb'))                        
        X_test = np.delete(X_test, 163, axis=1)
        

        #print features
        '''for m in range(len(X_test)):
            for feature, col in zip(X_test[m], cols):
                print(col, feature)'''
        X_test = X_test.astype(np.float32)
        # log transform

        X_test = np.log(X_test + 1)

        x_test = torch.from_numpy(X_test).to("cuda")
        # as float32
        x_test = x_test.float()
        test_dataset = torch.utils.data.TensorDataset(x_test)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)
        y_pred = []
        for batch_idx, (x_test) in enumerate(test_loader):
            x_test = x_test[0]
            x_recon, mu, logvar, z, classifications = vae_model(x_test)
            # apply sigmoid on classifications


            torch.cuda.empty_cache()
            classification = classifications[:, idx, :]
            #softmax, take first 2 logits
            classification = nn.Softmax(dim=1)(classification[:, :2])

            y_pred_ = np.argmax(classification.cpu().detach().numpy(), axis=1)
            y_pred.append(y_pred_)
        y_pred = np.concatenate(y_pred, axis=0)
        targets = torch.tensor(y_test).to("cuda")

        
        #load attributions
        atts_ = pkl.load(open('./interpret' + '/attributions_fold_'+ outcome + '/' + str(i) + '.pickle', 'rb'))
        atts_ = atts_[(y_test == y_pred)]
        atts.append(atts_)
    atts = np.concatenate(atts, axis=0)
    #get atts such that the target is 1

    #get feature importance
    atts = np.mean(np.abs(atts), axis=0)
    #get first 163 features
    atts = atts[:163]
    #get feature names
    cols = cols[:163]
    #sort features and get indices from most important to least
    #get probabilities of each feature
    atts = atts/np.sum(atts)
    sorted_indices = np.argsort(atts)
    #get top features from most important to least
    top_features = [cols[i] for i in sorted_indices[-10:]]

    print(top_features)
    #Plot top features
    import matplotlib.pyplot as plt
    plt.figure()
    plt.barh(top_features, atts[sorted_indices[-10:]])
    plt.xlabel('Feature Importance')
    plt.ylabel('Preoperative Features')
    plt.title('Top 10 Features for ' + outcome )
    plt.show()




In [ ]:
import json
from itertools import combinations
from numbers import Number
from random import shuffle
from collections import Counter
import torch
import time
import logging
import argparse
import os
import math
import random
import numpy as np
import pickle as pkl
from pandas import DataFrame
from sklearn.metrics import roc_auc_score, average_precision_score
from torch import nn

from dataloader import *
import argparse
from transform import *
from captum.attr import IntegratedGradients

with open('../../../inputs_new.pickle','rb') as handle:
    inputs = pkl.load(handle)
print(inputs)
cols = inputs.preops.columns
#get rid of first and last
cols = cols[1:-1]    
#change column name orlogid_encoded to orlogid
inputs.outcomes.rename(columns={'orlogid_encoded':'orlogid'}, inplace=True)
inputs.texts.rename(columns={'orlogid_encoded':'orlogid'}, inplace=True)
print(inputs.outcomes.columns)
folder = ('../preops_cv/')

outcomes = ['cardiac', 'AF','arrest', 'DVT_PE', 'post_aki_status', 'total_blood']
for idx, outcome in enumerate(outcomes):
    foldername = folder+'arrest'+'/'
    atts = []
    for i in range(5):
        X_train = pkl.load(open(foldername + 'X_train_' + str(i) + '.pickle', 'rb'))
        X_test = pkl.load(open(foldername + 'X_test_' + str(i) + '.pickle', 'rb'))

        ids_train = pkl.load(open('../preops_cv/arrest/' + 'train_ids_' + str(i) + '.pickle', 'rb'))
        ids_test = pkl.load(open('../preops_cv/arrest/' + 'test_ids_' + str(i) + '.pickle', 'rb'))
        train_outcome2 = inputs.outcomes[(inputs.outcomes.orlogid.isin(ids_train))].copy()
        train_outcome2 = train_outcome2[['cardiac', 'AF', 'arrest', 'DVT_PE', 'post_aki_status', 'total_blood']].to_numpy()
        train_outcome2 = np.where(train_outcome2> 0, 1, 0)
        test_outcome2 = inputs.outcomes[(inputs.outcomes.orlogid.isin(ids_test))].copy()
        test_outcome2 = test_outcome2[['cardiac', 'AF', 'arrest', 'DVT_PE', 'post_aki_status', 'total_blood']].to_numpy()
        test_outcome2 = np.where(test_outcome2> 0, 1, 0)
        y_train = train_outcome2[:, idx]
        y_test = test_outcome2[:, idx]

        print(outcome + ' ' + "fold " + str(i))
        vae_model = pkl.load(open('./distangle_vae_model_epoch/model_fold' + str(i) + '.pickle', 'rb'))                        
        X_test = np.delete(X_test, 163, axis=1)
        

        #print features
        '''for m in range(len(X_test)):
            for feature, col in zip(X_test[m], cols):
                print(col, feature)'''
        X_test = X_test.astype(np.float32)
        # log transform

        X_test = np.log(X_test + 1)

        x_test = torch.from_numpy(X_test).to("cuda")
        # as float32
        x_test = x_test.float()
        test_dataset = torch.utils.data.TensorDataset(x_test)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)
        y_pred = []
        for batch_idx, (x_test) in enumerate(test_loader):
            x_test = x_test[0]
            x_recon, mu, logvar, z, classifications = vae_model(x_test)
            # apply sigmoid on classifications


            torch.cuda.empty_cache()
            classification = classifications[:, idx, :]
            #softmax, take first 2 logits
            classification = nn.Softmax(dim=1)(classification[:, :2])

            y_pred_ = np.argmax(classification.cpu().detach().numpy(), axis=1)
            y_pred.append(y_pred_)
        y_pred = np.concatenate(y_pred, axis=0)
        targets = torch.tensor(y_test).to("cuda")

        
        #load attributions
        atts_ = pkl.load(open('./interpret' + '/attributions_fold_'+ outcome + '/' + str(i) + '.pickle', 'rb'))
        atts_ = atts_[(y_test == y_pred)]
        atts.append(atts_)
    atts = np.concatenate(atts, axis=0)
    #get atts such that the target is 1

    #get feature importance
    atts = np.mean(np.abs(atts), axis=0)
    #get first 163 features
    atts = atts[:163]
    #get feature names
    cols = cols[:163]
    #sort features and get indices from most important to least
    #get probabilities of each feature
    atts = atts/np.sum(atts)
    sorted_indices = np.argsort(atts)
    #get top features from most important to least
    top_features = [cols[i] for i in sorted_indices[-12:]]

    print(top_features)
    #store into csv
    df = DataFrame({'Feature': top_features, 'Importance': atts[sorted_indices[-12:]]})
    df.to_csv('./interpret' + '/top_features_fold_'+ outcome + '.csv', index=False)





In [ ]:
import json
from itertools import combinations
from numbers import Number
from random import shuffle
from collections import Counter
import torch
import time
import logging
import argparse
import os
import math
import random
import numpy as np
import pickle as pkl
from pandas import DataFrame
from sklearn.metrics import roc_auc_score, average_precision_score
from torch import nn

from dataloader import *
import argparse
from transform import *
from captum.attr import IntegratedGradients

with open('../../../inputs_new.pickle','rb') as handle:
    inputs = pkl.load(handle)
print(inputs)
cols = inputs.preops.columns
#get rid of first and last
cols = cols[1:-1]    
#change column name orlogid_encoded to orlogid
inputs.outcomes.rename(columns={'orlogid_encoded':'orlogid'}, inplace=True)
inputs.texts.rename(columns={'orlogid_encoded':'orlogid'}, inplace=True)
print(inputs.outcomes.columns)
folder = ('../preops_cv/')

outcomes = ['cardiac', 'AF','arrest', 'DVT_PE', 'post_aki_status', 'total_blood']
for idx, outcome in enumerate(outcomes):
    foldername = folder+'arrest'+'/'
    aurocs = []
    auprcs = []
    i = 1

    X_train = pkl.load(open(foldername + 'X_train_' + str(i) + '.pickle', 'rb'))
    X_test = pkl.load(open(foldername + 'X_test_' + str(i) + '.pickle', 'rb'))

    ids_train = pkl.load(open('../preops_cv/arrest/' + 'train_ids_' + str(i) + '.pickle', 'rb'))
    ids_test = pkl.load(open('../preops_cv/arrest/' + 'test_ids_' + str(i) + '.pickle', 'rb'))
    train_outcome2 = inputs.outcomes[(inputs.outcomes.orlogid.isin(ids_train))].copy()
    train_outcome2 = train_outcome2[['cardiac', 'AF', 'arrest', 'DVT_PE', 'post_aki_status', 'total_blood']].to_numpy()
    train_outcome2 = np.where(train_outcome2> 0, 1, 0)
    test_outcome2 = inputs.outcomes[(inputs.outcomes.orlogid.isin(ids_test))].copy()
    test_outcome2 = test_outcome2[['cardiac', 'AF', 'arrest', 'DVT_PE', 'post_aki_status', 'total_blood']].to_numpy()
    test_outcome2 = np.where(test_outcome2> 0, 1, 0)
    y_train = train_outcome2[:, idx]
    y_test = test_outcome2[:, idx]

    print(outcome + ' ' + "fold " + str(i))

    vae_model = pkl.load(open('./distangle_vae_model/model_fold' + str(i) + '.pickle', 'rb'))

    X_test = np.delete(X_test, 163, axis=1)
    i_list = [549]

    X_test = X_test[i_list]
    y_test = y_test[i_list]
    X_test = X_test.astype(np.float32)
    # log transform

    X_test = np.log(X_test + 1)

    vae_model = vae_model.to("cuda")

    #construct model using encoder of vae and classifier
    class VAE(nn.Module):
        def __init__(self, vae_model):
            super(VAE, self).__init__()
            self.feature_encoder = vae_model.feature_encoder
            self.encoder = vae_model.encoder
            self.classifier = vae_model.classifiers[idx]
        def forward(self, x):
            x = self.feature_encoder(x)
            x = self.encoder(x)
            x = self.classifier(x)
            #softmax
            x = nn.Softmax(dim=1)(x[:, :2])
            return x
    model = VAE(vae_model)
    model = model.to("cuda")
    model.train()

    ig = IntegratedGradients(model)
    x_test = torch.from_numpy(X_test).to("cuda")
    # as float32
    x_test = x_test.float()

    targets = torch.tensor(y_test).to("cuda")
    #use batch size of 64
    dataset = torch.utils.data.TensorDataset(x_test, targets)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=False)
    #use integrated gradients
    atts = []
    for batch in dataloader:
        x, y = batch
        x.requires_grad = True
        attributions = ig.attribute(x, target=y, n_steps=100)
        atts.append(attributions.cpu().detach().numpy())
        #release memory
        del x
        del y
        del attributions

    #release memory
    del x_test
    del targets
    del dataset
    del dataloader

    atts = np.concatenate(atts, axis=0)

    #save attributions
    if not os.path.exists('./interpret_single'):
        os.makedirs('./interpret_single')
    if not os.path.exists('./interpret_single' + '/attributions_fold_'+ outcome):
        os.makedirs('./interpret_single' + '/attributions_fold_'+ outcome)
    pkl.dump(atts, open('./interpret_single' + '/attributions_fold_'+ outcome + '/' + str(i) + '.pickle', 'wb'))
    atts = np.mean(np.abs(atts), axis=0)
    #get first 163 features
    atts = atts[:163]
    #get feature names
    cols = cols[:163]
    #sort features and get indices from most important to least
    #get probabilities of each feature
    atts = atts/np.sum(atts)
    sorted_indices = np.argsort(atts)
    #get top features from most important to least
    top_features = [cols[i] for i in sorted_indices[-12:]]
    print(top_features)

    #store into csv
    df = DataFrame({'Feature': top_features, 'Importance': atts[sorted_indices[-12:]]})
    df.to_csv('./interpret_single' + '/top_features_fold_'+ outcome + '.csv', index=False)
    
